In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime
import matplotlib.pyplot as plt
import sqlite3
from tqdm import tqdm
import pickle
import time 

from datetime import datetime
from datetime import timedelta

from create_financial_database import get_credentials 
from SQLite_tools import query_stock_data, check_if_close_price_exists
from ticker_loader import load_SPY_components

import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tqdm.notebook import tqdm
#import ImbalancedDatasetSampler

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
print("Expected Output :")
print("2.2.0+cu121")
print("CUDA available: True")
print("CUDA version: 12.1 \n")
print("Actual Output :")
print(torch.__version__)  
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda")
# DEVICE = torch.device("cpu")
print("CUDA:", USE_CUDA, DEVICE)
print("")
print("PyTorch Devices: ", torch.cuda.get_device_name(0))
print("Using GPU: ", USE_CUDA)
print("DEVICE: ", DEVICE)

Expected Output :
2.2.0+cu121
CUDA available: True
CUDA version: 12.1 

Actual Output :
2.2.0+cu121
CUDA available: True
CUDA version: 12.1
CUDA: True cpu

PyTorch Devices:  NVIDIA GeForce RTX 3060
Using GPU:  True
DEVICE:  cpu


In [ ]:
# Define the directory containing the chunks
# CHUNK_DATA_DIR = "../Data/Networks_chunks/window_size_10"
# CHUNK_DATA_DIR = "../Data/Networks_chunks/window_size_10"
# CHUNK_DATA_DIR = "../Data/Networks_chunks/window_size_3"
WINDOW_SIZE = 5
data_file_path = f"../Data/LSTM_Data/lstm_windowed_data_{WINDOW_SIZE}.pkl"
# data_file_path = "../Data/LSTM_Data/lstm_windowed_data_{WINDOW_SIZE}.pkl"


def load_windowed_data(file_path):
    with open(file_path, "rb") as f:
        lstm_dataset = pickle.load(f)
    return lstm_dataset


total_lstm_dataset = load_windowed_data(data_file_path)
total_lstm_dataset[-100]



{'instance_id': 2903577,
 'ticker': 'ZTS',
 'ticker_idx': 733,
 'window': array([[ 0.12254116,  0.25601934, -0.84055526, -0.76457537, -0.02506234,
         -0.02527723, -0.19047589, -0.19248162, -0.41871859, -0.32199907,
         -0.20232042, -0.07964685,  0.14715067, -0.12516569,  1.59445815,
         -0.52579677, -0.03342265,  0.26320189,  0.42490559,  0.35513859,
          0.49803776,  0.50527383, -0.17948227, -0.14147084,  0.59037396,
          0.24346065,  0.38853698, -0.60667225,  0.9887194 , -0.79836474,
         -0.16296185,  0.56846405, -0.08391866,  0.65376341,  0.24796584,
          0.19390044,  0.02787609,  0.11651947, -0.35853148,  0.18333564,
          0.21109882, -0.21741926, -0.04909748, -0.03888687,  0.7661333 ,
          0.91521042, -0.7180225 , -0.40299145, -0.24159425, -0.46659269,
         -0.27657878, -0.67801973, -0.97433974,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  1. 

In [ ]:
class LSTMDataset:
    """
    PyTorch Dataset class for creating train, validation, and test splits from windowed data.

    Args:
        windowed_data (list): List of dictionaries, each containing a window and metadata.
        val_date (str): The cutoff date for the validation set. Instances with `start_date` >= `val_date` go into the validation set.
        test_date (str): The cutoff date for the test set. Instances with `start_date` >= `test_date` go into the test set.
    """
    def __init__(self, windowed_data, val_date, test_date):
        self.windowed_data = windowed_data
        self.val_date = val_date
        self.test_date = test_date

    def _filter_split(self, split):
        """
        Filters the dataset based on the split and dates.

        Args:
            split (str): The split type ("train", "val", or "test").

        Returns:
            list: The filtered dataset for the specified split.
        """
        if split == "train":
            return [d for d in self.windowed_data if d['start_date'] < self.val_date]
        elif split == "val":
            return [d for d in self.windowed_data if self.val_date <= d['start_date'] < self.test_date]
        elif split == "test":
            return [d for d in self.windowed_data if d['start_date'] >= self.test_date]
        else:
            raise ValueError("Invalid split type. Choose from 'train', 'val', or 'test'.")

    def get_datasets(self):
        """
        Returns the train, validation, and test datasets.

        Returns:
            tuple: Three PyTorch datasets for train, val, and test splits.
        """
        train_data = self._filter_split("train")
        val_data = self._filter_split("val")
        test_data = self._filter_split("test")

        return (
            SingleSplitDataset(train_data),
            SingleSplitDataset(val_data),
            SingleSplitDataset(test_data)
        )


class SingleSplitDataset(Dataset):
    """
    PyTorch Dataset for a single split (train, val, or test).

    Args:
        split_data (list): List of dictionaries containing the split data.
    """
    def __init__(self, split_data):
        self.data = split_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instance = self.data[idx]
        window = torch.tensor(instance['window'], dtype=torch.float32) 
        target = torch.tensor(instance['target'], dtype=torch.float32)   # converting int to float to define the task as a regression problem
        prediction_date = instance['prediction_date']  
        ticker = instance['ticker']  
        return window, target, prediction_date, ticker


# I messed up the dates a bit, but it's close enough
val_date = "2016-11-30"
test_date ="2018-11-30"

lstm_dataset = LSTMDataset(total_lstm_dataset, val_date, test_date)
train_dataset, val_dataset, test_dataset = lstm_dataset.get_datasets()


for window_data_instace, target_instance, instance_pred_date, instance_ticker in test_dataset:
    print("instance_pred_date: ", instance_pred_date)
    print("instance_ticker:", instance_ticker)
    print("target_instance Shape:", target_instance.shape)
    print("target_instance:", target_instance)
    print("target_instance type:", type(target_instance))
    print("window_data_instace Shape:", window_data_instace.shape)
    print("window_data_instace:", window_data_instace)
    print("window_data_instace type:", type(window_data_instace))

    break

instance_pred_date:  2018-12-07
instance_ticker: A
target_instance Shape: torch.Size([])
target_instance: tensor(-1.)
target_instance type: <class 'torch.Tensor'>
window_data_instace Shape: torch.Size([5, 70])
window_data_instace: tensor([[ 0.0117,  0.5581, -0.1444,  0.3993,  0.0072,  0.0298, -0.2075, -0.0099,
          0.1391,  0.0674,  0.1739,  0.3312, -0.5357, -0.7263, -0.3017, -0.9592,
         -0.1857,  0.0961,  0.3064,  0.2900,  0.3800,  0.4602, -0.2640, -0.3817,
          0.5314, -0.2563, -0.6953,  1.2823,  0.4326,  1.2880, -0.1897, -1.1853,
         -0.3664, -0.7594, -0.8016, -0.7424,  0.3298,  0.1184,  0.8792,  0.0798,
          0.0100,  0.2867, -0.1009, -0.1249, -1.5258, -0.8229,  1.4637,  1.1086,
          1.3019,  0.8082, -0.2891, -0.6748, -0.6760,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.5011,  1.2787,  0.4266,  1.2148,  0.0072,  0.

In [ ]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch_window, batch_targets, batch_dates, batch_ticker in train_loader:
    print("Batch Dates:", batch_dates)  
    print("Batch Windows Shape:", batch_window.shape)  
    print("Batch Targets Shape:", batch_targets.shape)  
    print("Batch Targets:", batch_targets) 
    print("Batch Tickers Shape:", len(batch_ticker))  
    print("Batch Tickers:", batch_ticker)  

    break

Batch Dates: ('1999-10-08', '1996-06-13', '2000-06-08', '2001-12-27', '2006-12-14', '2003-11-26', '2007-10-26', '2014-07-23', '2005-02-10', '2006-02-27', '2007-08-07', '2004-01-20', '2009-06-29', '2005-10-24', '2014-12-12', '2004-09-14')
Batch Windows Shape: torch.Size([16, 5, 70])
Batch Targets Shape: torch.Size([16])
Batch Targets: tensor([-2., -2.,  0., -1.,  0.,  1.,  1.,  0.,  0., -1., -2.,  0., -2.,  1.,
         0.,  0.])
Batch Tickers Shape: 16
Batch Tickers: ('IFF', 'BAC', 'LEG', 'WMB', 'FMC', 'SEE', 'CNP', 'EBAY', 'PX', 'DOV', 'FE', 'ETR', 'CHRW', 'PSA', 'RAI', 'CCI')


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.0):
        """
        Args:
            input_size (int): Number of features per time step (e.g., 70 for this dataset).
            hidden_size (int): Number of features in the hidden state.
            num_layers (int): Number of stacked LSTM layers.
            output_size (int): Size of the output (e.g., 1 for scalar regression).
            dropout (float): Dropout probability for each connection between LSTM layers (0.0 for no dropout).
        """
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,      
            hidden_size=hidden_size,     
            num_layers=num_layers,       
            batch_first=True,       
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(p=dropout)  
        self.fc = nn.Linear(hidden_size, output_size)  # Fully connected output layer

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # Shape: (batch_size, sequence_length, hidden_size)
        # Use the last time step's hidden state
        last_hidden_state = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_size)
        # Apply dropout
        last_hidden_state = self.dropout(last_hidden_state)
        # Fully connected layer for output
        output = self.fc(last_hidden_state)  
        return output.squeeze()



In [7]:
def evaluate(model, dataloader, criterion, device, save_pred = False):
    """
    Evaluates the model.

    Args:
        model (nn.Module): The LSTM model.
        dataloader (DataLoader): DataLoader for the validation or test data.
        criterion: Loss function.
        device: Device to run the model on ("cpu" or "cuda").

    Returns:
        tuple: Average loss and predictions.
    """
    model.eval()
    total_loss = 0
    predictions = []
    targets = []
    tickers = []
    with torch.no_grad():
        for batch_windows, batch_targets, batch_dates, batch_tickers in dataloader:
            batch_windows, batch_targets = batch_windows.to(device), batch_targets.to(device)
            outputs = model(batch_windows)
            loss = criterion(outputs, batch_targets)

            total_loss += loss.item()
            if save_pred:
                predictions.extend(outputs.cpu().numpy())
                targets.extend(batch_targets.cpu().numpy())
                tickers.extend(batch_tickers)
    if save_pred:
        return total_loss / len(dataloader), predictions, targets, tickers
    else:
        return total_loss / len(dataloader)
    

In [8]:
class WeightedMSELoss(nn.Module):
    def __init__(self, weights=None):
        super(WeightedMSELoss, self).__init__()
        self.weights = weights if weights else {-2: 1, -1: 1.0, 0: 1.0, 1: 1.0, 2: 1.0}
        self.base_loss = nn.MSELoss()

    def forward(self, predictions, targets):
        device = predictions.device
        target_weights = torch.tensor([self.weights[int(target.item())] for target in targets], device=device)

        mse_loss = self.base_loss(predictions, targets)
        too_low_prediction_penalty = torch.zeros_like(predictions)
        too_high_prediction_penalty = torch.zeros_like(predictions)

        high_target_mask = (targets >= 1)
        too_low_prediction_penalty[high_target_mask] = (predictions[high_target_mask] < targets[high_target_mask]).float() * 1.5
        too_high_prediction_penalty[high_target_mask] = (predictions[high_target_mask] > targets[high_target_mask]).float() * 0.75

        low_target_mask = (targets == -2)
        too_low_prediction_penalty[low_target_mask] = (predictions[low_target_mask] > targets[low_target_mask]).float() * 1.5
        too_high_prediction_penalty[low_target_mask] = (predictions[low_target_mask] < targets[low_target_mask]).float() * 0.75

        penalties = 1 + too_low_prediction_penalty - too_high_prediction_penalty
        weighted_loss = mse_loss * target_weights * penalties
        return weighted_loss.mean()


In [ ]:
# Reload the model at the last state from the checkpoint IF the notebook crashed or was stopped for some reason.

# model_name = "LSTM_4"
# checkpoint = torch.load(f"../Data/Models/{model_name}.pt")
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# # best_val_loss = checkpoint['best_val_loss']
# model = model.to(DEVICE)

In [ ]:
# Training Loop Parameters
epochs = 500
patience = 4
force_epochs = 10
stop_training = False

learning_rate = 0.0000001
gamma = 0.90 
hidden_size = 128      
drop_out_rate = 0.5
weight_dict = {-2: 3.0, -1: 2.0, 0: 0.5, 1: 2.0, 2: 5.0} 
# weight_dict = {-2: 2.0, -1: 0.75, 0: 0.25 , 1: 3.0, 2: 8.0}
num_layers = 3                             

best_val_loss = float("inf")
model_name = "LSTM_4"
cur_patience = 0
output_size = 1                                 # Single prediction per instance
input_size = window_data_instace.shape[1]  #70  # Number of features per time step

print("Input Size:", input_size)
print("Hidden Size:", hidden_size)
print("Number of Layers:", num_layers)
print("Output Size:", output_size)

# Model, criterion, optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size, drop_out_rate).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion            = WeightedMSELoss(weights=weight_dict)
criterion_unweighted = WeightedMSELoss()

print(model)


Input Size: 70
Hidden Size: 128
Number of Layers: 3
Output Size: 1
LSTMModel(
  (lstm): LSTM(70, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


In [ ]:

train_losses, val_losses = [], []
train_unweighted_losses, val_unweighted_losses = [], []

# Model Directory
if not os.path.exists("../Data/Models/"):
    os.makedirs("../Data/Models/")

# Early stopping model path
model_path = f"../Data/Models/{model_name}.pt"
model_train_start_time = time.time()

# Training Loop
for epoch in range(epochs):
    total_loss = 0
    total_unweighted_loss = 0
    model.train()

    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{epochs}", unit="batch") as progress_bar:
        for batch_windows, batch_targets, batch_dates, batch_tickers in train_loader:
            batch_windows, batch_targets = batch_windows.to(DEVICE), batch_targets.to(DEVICE)
            
            # assert not torch.isnan(predictions).any(), "Predictions contain NaN values!"
            # assert not torch.isinf(predictions).any(), "Predictions contain Inf values!"
            # assert not torch.isnan(batch_targets).any(), "Targets contain NaN values!"
            # assert not torch.isinf(batch_targets).any(), "Targets contain Inf values!"

            # print("batch_windows shape:", batch_windows.shape)
            # print("batch_targets shape:", batch_targets.shape)
            # print("batch_targets:", batch_targets)

            predictions = model(batch_windows)
            # predictions = torch.nan_to_num(predictions, nan=0.0)

            # print("predictions shape:", predictions.shape)
            # print("predictions:", predictions)


            # These lines are here because I had a bug :)
            if torch.isnan(predictions).any():
                print("Some predictions are NaN, skipping batch.")
                continue
            if torch.isnan(predictions).all():
                print("All predictions are NaN, skipping batch.")
                continue

            loss = criterion(predictions, batch_targets)
            unweighted_loss = nn.MSELoss()(predictions, batch_targets)  # Compute unweighted MSE for logging

            optimizer.zero_grad()
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping (not necessary for this model)
            optimizer.step()

            total_loss += loss.item()
            total_unweighted_loss += unweighted_loss.item()

            # I missed tensorflows progress bar, so I made my own which kind of works. D tier on visuals, B tier on effeciecy.
            progress_bar.set_postfix({"Training Loss": f"{total_loss / (progress_bar.n + 1):.4f}"})  
            progress_bar.update(1) 


    train_loss = total_loss / len(train_loader)
    train_unweighted_loss = total_unweighted_loss / len(train_loader)


    model.eval()
    val_loss = evaluate(model, val_loader, criterion, DEVICE)
    val_unweighted_loss = evaluate(model, val_loader, criterion_unweighted, DEVICE)

    train_losses.append(train_loss)
    train_unweighted_losses.append(train_unweighted_loss)
    val_losses.append(val_loss)
    val_unweighted_losses.append(val_unweighted_loss)

    # Early stopping / patience block
    force_epochs -= 1
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        cur_patience = 0
        torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, model_path)
    else:
        if force_epochs <= 0:
            cur_patience += 1
            if cur_patience == patience:
                print(f"Early stopping: validation loss did not improve for {patience} epochs")
                break
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate * (gamma ** epoch)

    # Logging progress
    print(f"Epoch {epoch + 1}/{epochs}   Current Learning Rate: {param_group['lr']}   Training Loss Weighted MSE: {train_loss:.4f}    "
          f"Validation Loss Weighted MSE: {val_loss:.4f} | Unweighted MSE: {val_unweighted_loss:.4f}")


NameError: name 'os' is not defined

In [ ]:
def final_model_evaluation(model, test_loader, val_loader, 
                            model_name, criterion, criterion_unweighted, 
                            optimizer, train_losses, train_unweighted_losses, 
                            val_losses, val_unweighted_losses, patience, epoch, start_time):
    """
    Evaluates the LSTM model on the test and validation sets.

    Args:
        model (nn.Module): The trained LSTM model to evaluate.
        test_loader (DataLoader): DataLoader for the test dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        test_files (list): List of test files used to create predictions.
        model_name (str): Name of the model for saving results.
        criterion (nn.Module): Weighted loss function.
        criterion_unweighted (nn.Module): Unweighted loss function.
        optimizer (torch.optim.Optimizer): Optimizer used in training.
        train_losses (list): List of training losses per epoch.
        train_unweighted_losses (list): List of unweighted training losses per epoch.
        val_losses (list): List of validation losses per epoch.
        val_unweighted_losses (list): List of unweighted validation losses per epoch.
        patience (int): Patience value for early stopping.
        epoch (int): Number of training epochs completed.

    Returns:
        None
    """
    model.eval()  
    predictions = []
    val_predictions = []
    ground_truth = []
    val_ground_truth = []
    date_of_test_predictions = []
    date_of_val_predictions = []
    tickers_of_test_predictions = []
    tickers_of_val_predictions = []
    total_test_loss = 0
    total_val_loss = 0
    total_unweighted_test_loss = 0
    total_unweighted_val_loss = 0

    with torch.no_grad():  
        for X_batch, y_batch, y_batch_dates, y_batch_tickers in test_loader:
            date_of_test_predictions.extend(y_batch_dates)
            tickers_of_test_predictions.extend(y_batch_tickers)
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            batch_predictions = model(X_batch).squeeze(-1)  
            predictions.extend(batch_predictions.cpu().numpy())
            ground_truth.extend(y_batch.cpu().numpy())
            total_test_loss += criterion(batch_predictions, y_batch.float()).item()
            total_unweighted_test_loss += criterion_unweighted(batch_predictions, y_batch.float()).item()


        for X_batch, y_batch, y_batch_dates, y_batch_tickers in val_loader:
            date_of_val_predictions.extend(y_batch_dates)
            tickers_of_val_predictions.extend(y_batch_tickers)
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            val_batch_predictions = model(X_batch).squeeze(-1) 
            val_predictions.extend(val_batch_predictions.cpu().numpy())
            val_ground_truth.extend(y_batch.cpu().numpy())
            total_val_loss += criterion(val_batch_predictions, y_batch.float()).item()
            total_unweighted_val_loss += criterion_unweighted(val_batch_predictions, y_batch.float()).item()


    test_loss = total_test_loss / len(test_loader)
    test_unweighted_loss = total_unweighted_test_loss / len(test_loader)
    val_loss = total_val_loss / len(val_loader)
    val_unweighted_loss = total_unweighted_val_loss / len(val_loader)

    # THis is the results dictionary that was fucky on the ATGCN 1,2 and 3
    results_dict = {
        "Test_Dates": date_of_test_predictions,
        "Prediction": predictions,
        "Ground Truth": ground_truth,
        "Tickers": tickers_of_test_predictions,
        "Val_Dates": date_of_val_predictions,
        "Val_Prediction": val_predictions,
        "Val_Ground Truth": val_ground_truth,
        "Val_Tickers": tickers_of_val_predictions,
        "Model_Name": model_name,
        "Hidden_Dim": model.lstm.hidden_size,  
        "Num_Layers": model.lstm.num_layers,  
        "Batch_Size": test_loader.batch_size,
        "Optimizer": optimizer.__class__.__name__,
        "Learning_Rate": optimizer.param_groups[0]['lr'],
        "Gamme (LR Decay)": gamma,
        "Loss_Function": "WeightedMSELoss",
        "Loss_Function_Weights": criterion.weights,
        "Dropout": drop_out_rate,
        "Epochs": epoch,
        "Train_Time": str(time.time() - start_time),
        "Patience": patience,
        "Train_Loss": train_losses,
        "Train_Unweighted_Loss": train_unweighted_losses,
        "Val_Loss": val_losses,
        "Val_Unweighted_Loss": val_unweighted_losses,
        "Test_Loss": test_loss,
        "Test_Unweighted_Loss": test_unweighted_loss
    }

    with open(f"../Data/Results/{model_name}_results.pkl", 'wb') as f:
        pickle.dump(results_dict, f)

    print(f"Results saved to ../Data/Results/{model_name}_results.pkl")


final_model_evaluation(
    model=model,
    test_loader=test_loader,
    val_loader=val_loader,
    model_name=model_name,
    criterion=criterion,
    criterion_unweighted=criterion_unweighted,
    optimizer=optimizer,
    train_losses=train_losses,
    train_unweighted_losses=train_unweighted_losses,
    val_losses=val_losses,
    val_unweighted_losses=val_unweighted_losses,
    patience=patience,
    epoch=epoch,
    start_time=model_train_start_time
)


Results saved to ../Data/Results/LSTM_4_results.pkl
